# マルチエージェントシステムのアーキテクチャーを紐解く

## 事前準備

In [2]:
import copy, os, uuid
import vertexai

from google.genai.types import Part, Content
from google.adk.agents.llm_agent import LlmAgent
from google.adk.artifacts import InMemoryArtifactService
from google.adk.memory.in_memory_memory_service import InMemoryMemoryService
from google.adk.sessions import InMemorySessionService
from google.adk.runners import Runner

from google.adk.agents.callback_context import CallbackContext
from google.adk.models import LlmResponse, LlmRequest
from google.adk.agents.sequential_agent import SequentialAgent

[PROJECT_ID] = !gcloud config list --format 'value(core.project)'
LOCATION = 'us-central1'

vertexai.init(project=PROJECT_ID, location=LOCATION,
              staging_bucket=f'gs://{PROJECT_ID}')

os.environ['GOOGLE_CLOUD_PROJECT'] = PROJECT_ID
os.environ['GOOGLE_CLOUD_LOCATION'] = LOCATION
os.environ['GOOGLE_GENAI_USE_VERTEXAI'] = 'True'

In [3]:
class LocalApp:
    def __init__(self, agent, app_name='default_app', user_id='default_user'):
        self._agent = agent
        self._app_name = app_name
        self._user_id = user_id
        self._runner = Runner(
            app_name=self._app_name,
            agent=self._agent,
            artifact_service=InMemoryArtifactService(),
            session_service=InMemorySessionService(),
            memory_service=InMemoryMemoryService(),
        )
        self._session = None
        
    async def stream(self, query):
        if not self._session:
            self._session = await self._runner.session_service.create_session(
                app_name=self._app_name,
                user_id=self._user_id,
                session_id=uuid.uuid4().hex,
            )
        content = Content(role='user', parts=[Part(text=query)])
        async_events = self._runner.run_async(
            user_id=self._user_id,
            session_id=self._session.id,
            new_message=content,
        )
        result = []
        async for event in async_events:
            if DEBUG:
                print(f'====\n{event}\n====')
            if (event.content and event.content.parts):
                response = '\n'.join([p.text for p in event.content.parts if p.text])
                if response:
                    print(response)
                    result.append(response)
        return result

In [4]:
def get_print_agent(text):
    def before_model_callback(
        callback_context: CallbackContext, llm_request: LlmRequest
    ) -> LlmResponse:
         return LlmResponse(
            content=Content(
                role='model', parts=[Part(text=text)],
            )
        )
        
    return LlmAgent(
        name='print_agent',
        model='gemini-2.0-flash', # not used
        description='',
        instruction = '',
        before_model_callback=before_model_callback,
    )

## リサーチエージェントの定義

In [5]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
指定されたテーマの記事を執筆する際に参考となるトピックを５項目程度のリストにまとめます。
後段のエージェントがこのリストに基づいて、調査レポートを作成します。

* 出力形式
日本語で出力。
'''

research_agent1 = LlmAgent(
    name='research_agent1',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（テーマ選定）',
    instruction=instruction,
)

In [6]:
instruction = '''
あなたの役割は、記事の執筆に必要な情報を収集して調査レポートにまとめる事です。
前段のエージェントは、５項目程度の調査対象トピックを指定します。

* 出力形式
日本語で出力。
調査レポートは、トピックごとに客観的情報をまとめます。各トピックについて、５文以上の長さで記述すること。
'''

research_agent2 = LlmAgent(
    name='research_agent2',
    model='gemini-2.5-flash',
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント（レポート作成）',
    instruction=instruction,
)

In [8]:
research_agent = SequentialAgent(
    name='research_agent',
    sub_agents=[
        get_print_agent('\n---\n## リサーチエージェントが調査レポートを作成します。\n---\n'),
        get_print_agent('\n## 調査対象のトピックを選定します。\n'),
        copy.deepcopy(research_agent1),
        get_print_agent('\n## 選定したトピックに基づいて、調査レポートを作成します。\n'),
        copy.deepcopy(research_agent2),
        get_print_agent('\n#### 調査レポートが準備できました。記事の作成に取り掛かってもよいでしょうか？\n'),
    ],
    description='記事の執筆に必要な情報を収集してレポートにまとめるエージェント',
)

## ライターエージェント、レビューエージェントの定義

In [9]:
instruction = '''
あなたの役割は、特定のテーマに関する気軽な読み物記事を書くことです。
記事の「テーマ」と、その内容に関連する「調査レポート」が与えられるので、
調査レポートに記載の客観的事実に基づいて、信頼性のある読み物記事を書いてください。

レビュアーによる修正ポイントが指摘された際は、直前に書いた記事を指摘に従って書き直してください。

**出力条件**
- トピックに関してある程度の基礎知識がある読者を前提として、数分で気軽に読める内容にしてください。
- 比較的カジュアルで語りかける口調の文章にします。
- 思考過程は出力せずに、最終結果だけを出力します。
- 記事タイトルは付けないで、次の構成で出力します。各セクションタイトルは、内容に合わせてアレンジしてください。
0. 導入：セクションタイトルを付けないで、この記事を読みたくなる導入を１〜２文にまとめます。
1. 概要：トピックの全体像をまとめて簡単に説明します。
2. 最新情報：特に注目したい新しい情報を取り上げます。
3. 実践：トピックに関して、読者自身がやってみるとよさそうな事を１つ紹介します。
4. まとめ

- 各セクションのタイトルはマークダウンヘッダ ## 付けます。必要に応じて箇条書きのマークダウンを使用します。
- それ以外のマークダウンによる装飾は行いません。
'''

writer_agent = LlmAgent(
    name='writer_agent',
    model='gemini-2.5-flash',
    description='特定のテーマに関する読み物記事を書くエージェント',
    instruction=instruction,
)

In [10]:
instruction = f'''
あなたの役割は、読み物記事をレビューして、記事の条件にあった内容にするための改善コメントを与える事です。

* 記事の条件
- 記事は、はじめに４０文字程度のタイトルがあること。
　今日から役立つ生活情報があって「すぐに読まなきゃ」と読者が感じるタイトルにすること。
　タイトルはマークダウンヘッダ # をつけること。
- タイトルの直後に「なぜいまこのテーマを取り上げるのか」をまとめた導入を加えて、読者にこの記事を読む動機づけを与えます。
- 各セクションのサブタイトルには、絵文字を用いて親しみやすさを出すこと。
- 読者が今日から実践できる具体例が３つ以上紹介されていること。

* 出力形式
- 日本語で出力。
- はじめに、記事の良い点を説明します。
- 次に、修正ポイントを箇条書きで出力します。
'''

review_agent = LlmAgent(
    name='review_agent',
    model='gemini-2.5-flash',
    description='読み物記事をレビューするエージェント',
    instruction=instruction,
)

In [11]:
write_and_review_agent = SequentialAgent(
    name='write_and_review_agent',
    sub_agents=[
        get_print_agent('\n---\n## ライターエージェントが記事を執筆します。\n---\n'),
        copy.deepcopy(writer_agent),
        get_print_agent('\n---\n## レビューエージェントが記事をレビューします。\n---\n'),
        copy.deepcopy(review_agent),
       get_print_agent('\n#### レビューに基づいて記事の修正を依頼しますか？\n'),
    ],
    description='記事を作成、レビューする。',
)

## 業務フローエージェントの定義

In [12]:
root_agent = LlmAgent(
    name='article_generation_flow',
    model='gemini-2.0-flash',
    instruction = '''
何ができるか聞かれた場合は、以下の処理をすることをわかりやすくフレンドリーな文章にまとめて返答してください。

- ユーザーが指定したテーマの記事を作成する業務フローを実行する。
- はじめに、テーマに関する調査レポートを作成する。
- その後、ライターエージェントとレビューエージェントが協力して、編集方針に則した記事を作成する。

ユーザーが記事のテーマを指定した場合は、次のフローを実行します。

1. そのテーマの記事の作成に取り掛かる旨を伝えて、research_agent に転送して、調査レポートを依頼します。
2. ユーザー記事の作成を支持したら、write_and_review_agent に転送して、記事の作成とレビューを依頼します。
3. ユーザーが記事の修正を希望する場合は、write_and_review_agent に転送します。

**条件**
research_agent のニックネームは、リサーチエージェント
write_and_review_agent のネックネームは、ライターエージェントとレビューエージェント

''',
    sub_agents=[
        copy.deepcopy(research_agent),
        copy.deepcopy(write_and_review_agent),
    ],
    description='記事を作成する業務フローを実行するエージェント'
)

## 実行例

In [13]:
client = LocalApp(root_agent)

DEBUG=False
query = '''
こんにちは。何ができますか？
'''
result = await client.stream(query)

こんにちは！私は、お客様が指定されたテーマに関する記事を作成するお手伝いをすることができます。

具体的には、

1.  まず、リサーチエージェントがテーマに関する調査レポートを作成します。
2.  次に、ライターエージェントとレビューエージェントが協力して、編集方針に沿った記事を作成します。

記事のテーマを指定していただければ、すぐに作成に取り掛かります。リサーチエージェントに調査レポートを依頼しますか？あるいは、記事の作成を直接ライターエージェントとレビューエージェントに依頼することも可能です。修正が必要な場合も、ライターエージェントとレビューエージェントが対応します。ご希望をお聞かせください！



In [14]:
query = f'''
夏を乗り切る郷土料理のスパイシーアレンジ
'''
result = await client.stream(query)

夏を乗り切る郷土料理のスパイシーアレンジの記事ですね！承知いたしました。

まずは、リサーチエージェントにこのテーマに関する調査レポートを作成してもらうように依頼します。よろしいでしょうか？



In [15]:
query = f'''
はい。お願いします。
'''
result = await client.stream(query)


---
## リサーチエージェントが調査レポートを作成します。
---


## 調査対象のトピックを選定します。

承知いたしました。「夏を乗り切る郷土料理のスパイシーアレンジ」に関する調査レポートを作成するためのトピックを以下に選定します。

---

### 調査対象のトピックリスト

1.  **夏バテ防止に効果的な辛味のメカニズムとスパイスの選び方**
    *   夏に辛いものが好まれる理由（発汗作用、食欲増進、疲労回復効果など）
    *   夏バテ対策に特に有効とされるスパイス（唐辛子、胡椒、生姜、ニンニク、ハーブなど）とその栄養学的側面。
2.  **日本各地の「夏を乗り切る」郷土料理の具体例**
    *   地域ごとに伝わる、夏場に食される代表的な郷土料理（冷汁、ひやむぎ、鶏飯、豚汁、ゴーヤチャンプルーなど、元々スパイシーでないものも含む）。
    *   これらの料理が夏に選ばれる理由（栄養価、食べやすさ、涼感、滋養強壮など）。
3.  **郷土料理を「スパイシーアレンジ」する際の具体的なアイデアと成功事例**
    *   既存の郷土料理に辛味を加える手法（スパイスの追加、辛味調味料の利用、調理法の変更、ハーブとの組み合わせなど）。
    *   既にスパイシーにアレンジされ、人気を博している郷土料理や類似の事例。
4.  **地域性を活かしたスパイシーアレンジの可能性と調和**
    *   各地域の特産スパイスや辛味調味料（柚子胡椒、七味唐辛子、琉球唐辛子、かんずりなど）を活用したアレンジ方法。
    *   郷土料理本来の風味を損なわず、辛味を加えるためのバランスの取り方。
5.  **スパイシーアレンジ郷土料理の食欲増進・健康効果**
    *   辛味を加えることで、夏場の食欲不振を解消し、栄養摂取を促進する効果。
    *   郷土料理の持つ栄養価とスパイスの健康効果を組み合わせた、夏を健康的に乗り切るための提案。

---

これらのトピックに基づき、リサーチエージェントが詳細な調査を行います。

## 選定したトピックに基づいて、調査レポートを作成します。

「夏を乗り切る郷土料理のスパイシーアレンジ」に関する調査レポートを以下にまとめます。

---

### 調査レポート：夏を乗り切る郷土料理のスパ

In [16]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)


---
## ライターエージェントが記事を執筆します。
---

じめじめ暑い日本の夏。食欲が落ちたり、なんだか体がだるい…なんてこと、ありませんか？そんな時に恋しくなるのが、ピリッと辛い料理ですよね。実は、昔から日本には夏を乗り切るための知恵が詰まった郷土料理がたくさんあるんです。今回は、そんな郷土料理をちょっとスパイシーにアレンジして、元気に夏を乗り切るヒントを探ってみましょう！

## 郷土料理と辛味の素敵な関係

日本の郷土料理には、古くからその土地の気候や風土に合わせた知恵が詰まっています。夏バテしやすい時期には、消化に優しいもの、栄養価が高いもの、そして食欲をそそる工夫が凝らされてきました。例えば、宮崎県の「冷汁」や鹿児島県の「鶏飯」のように、のど越しが良く、さらさらと食べられる料理は、食欲が落ちた時でも栄養を摂りやすいようにと工夫されたものです。

一方、辛味はというと、唐辛子や胡椒に含まれるカプサイシンなどの成分が、唾液や胃液の分泌を促して消化を助け、食欲を増進させる効果が期待できます。また、発汗作用で体温調節を促したり、血行を良くして新陳代謝を高めたりする働きもあるので、まさに夏バテ対策にはもってこいなんです。郷土料理が持つ滋養強壮の効果に、スパイスのパワーが加われば、鬼に金棒というわけですね。

## 進化する！地域性を活かしたスパイシーアレンジ

ただ辛くするだけじゃないのが、最近のスパイシーアレンジの面白いところ。今注目されているのは、その郷土料理が生まれた地域の特性を活かした辛味の加え方なんです。

たとえば、九州の爽やかな香りの「柚子胡椒」、新潟のまろやかな旨味を持つ発酵調味料「かんずり」、沖縄のパワフルな「琉球唐辛子」や「コーレーグース」など、日本各地にはその土地ならではの辛味調味料があります。これらを郷土料理に加えることで、単に辛くなるだけでなく、より深みのある、その土地らしい風味が生まれるんですよ。郷土料理の出汁や素材の味を邪魔せず、辛味を「引き立て役」として活用するバランス感覚が、美味しいスパイシーアレンジの鍵になるんですね。

## 冷汁にちょっと冒険！マイ・スパイシーアレンジに挑戦

さあ、あなたも郷土料理のスパイシーアレンジに挑戦してみませんか？おすすめは、宮崎県の夏の定番「冷汁」です。

ご家庭で作った冷汁、または市販の

In [17]:
query = '''
はい。お願いします。
'''
result = await client.stream(query)


---
## ライターエージェントが記事を執筆します。
---

# 【夏バテ知らず！】郷土料理が激変！？ピリ辛アレンジで食欲UPの秘訣

じめじめ暑い日本の夏。今年は特に厳しい暑さになると言われていますよね。食欲が落ちたり、なんだか体がだるい…なんてこと、ありませんか？そんな時に恋しくなるのが、ピリッと辛い料理。実は、昔から日本には夏を乗り切るための知恵が詰まった郷土料理がたくさんあるんです。今回は、そんな郷土料理をちょっとスパイシーにアレンジして、元気に夏を乗り切るヒントを探ってみましょう！

## 郷土料理と辛味の素敵な関係🌶️

日本の郷土料理には、古くからその土地の気候や風土に合わせた知恵が詰まっています。夏バテしやすい時期には、消化に優しいもの、栄養価が高いもの、そして食欲をそそる工夫が凝らされてきました。例えば、宮崎県の「冷汁」や鹿児島県の「鶏飯」のように、のど越しが良く、さらさらと食べられる料理は、食欲が落ちた時でも栄養を摂りやすいようにと工夫されたものです。

一方、辛味はというと、唐辛子や胡椒に含まれるカプサイシンなどの成分が、唾液や胃液の分泌を促して消化を助け、食欲を増進させる効果が期待できます。また、発汗作用で体温調節を促したり、血行を良くして新陳代謝を高めたりする働きもあるので、まさに夏バテ対策にはもってこいなんです。郷土料理が持つ滋養強壮の効果に、スパイスのパワーが加われば、鬼に金棒というわけですね。

## 進化する！地域性を活かしたスパイシーアレンジ✨

ただ辛くするだけじゃないのが、最近のスパイシーアレンジの面白いところ。今注目されているのは、その郷土料理が生まれた地域の特性を活かした辛味の加え方なんです。

たとえば、九州の爽やかな香りの「柚子胡椒」、新潟のまろやかな旨味を持つ発酵調味料「かんずり」、沖縄のパワフルな「琉球唐辛子」や「コーレーグース」など、日本各地にはその土地ならではの辛味調味料があります。これらを郷土料理に加えることで、単に辛くなるだけでなく、より深みのある、その土地らしい風味が生まれるんですよ。郷土料理の出汁や素材の味を邪魔せず、辛味を「引き立て役」として活用するバランス感覚が、美味しいスパイシーアレンジの鍵になるんですね。

## 今日からできる！マイ・スパイシーアレンジに挑戦🥄

さあ、あなたも郷土料理の

In [18]:
query = f'''
これで完成です。完成した記事をもう一度表示して。
'''
result = await client.stream(query)

# 【夏バテ知らず！】郷土料理が激変！？ピリ辛アレンジで食欲UPの秘訣

じめじめ暑い日本の夏。今年は特に厳しい暑さになると言われていますよね。食欲が落ちたり、なんだか体がだるい…なんてこと、ありませんか？そんな時に恋しくなるのが、ピリッと辛い料理。実は、昔から日本には夏を乗り切るための知恵が詰まった郷土料理がたくさんあるんです。今回は、そんな郷土料理をちょっとスパイシーにアレンジして、元気に夏を乗り切るヒントを探ってみましょう！

## 郷土料理と辛味の素敵な関係🌶️

日本の郷土料理には、古くからその土地の気候や風土に合わせた知恵が詰まっています。夏バテしやすい時期には、消化に優しいもの、栄養価が高いもの、そして食欲をそそる工夫が凝らされてきました。例えば、宮崎県の「冷汁」や鹿児島県の「鶏飯」のように、のど越しが良く、さらさらと食べられる料理は、食欲が落ちた時でも栄養を摂りやすいようにと工夫されたものです。

一方、辛味はというと、唐辛子や胡椒に含まれるカプサイシンなどの成分が、唾液や胃液の分泌を促して消化を助け、食欲を増進させる効果が期待できます。また、発汗作用で体温調節を促したり、血行を良くして新陳代謝を高めたりする働きもあるので、まさに夏バテ対策にはもってこいなんです。郷土料理が持つ滋養強壮の効果に、スパイスのパワーが加われば、鬼に金棒というわけですね。

## 進化する！地域性を活かしたスパイシーアレンジ✨

ただ辛くするだけじゃないのが、最近のスパイシーアレンジの面白いところ。今注目されているのは、その郷土料理が生まれた地域の特性を活かした辛味の加え方なんです。

たとえば、九州の爽やかな香りの「柚子胡椒」、新潟のまろやかな旨味を持つ発酵調味料「かんずり」、沖縄のパワフルな「琉球唐辛子」や「コーレーグース」など、日本各地にはその土地ならではの辛味調味料があります。これらを郷土料理に加えることで、単に辛くなるだけでなく、より深みのある、その土地らしい風味が生まれるんですよ。郷土料理の出汁や素材の味を邪魔せず、辛味を「引き立て役」として活用するバランス感覚が、美味しいスパイシーアレンジの鍵になるんですね。

## 今日からできる！マイ・スパイシーアレンジに挑戦🥄

さあ、あなたも郷土料理のスパイシーアレンジに挑戦してみませんか？ここでは、いくつか手軽に試せ